In [1]:
from gensim.models import Word2Vec
from collections import Counter
import pickle

In [ ]:
def read_file(dataset_en, dataset_fr):
    #read the files
    with open(dataset_en) as f:
        sentences_en = [l.strip() for l in f.readlines()]
    
    with open(dataset_fr) as f:
        sentences_fr = [l.strip() for l in f.readlines()]

    sentence_list_en = []
    sentence_list_fr = []
    
    no_sentences = len(sentences_en)
    
    for s in range(no_sentences):
        
        sent_en = sentences_en[s]
        sent_fr = sentences_fr[s]
        
        split_sent_en = sent_en.split()
        split_sent_fr = sent_fr.split()
        
        if len(split_sent_en) <= 50 and len(split_sent_fr) <= 50:
            
            sentence_list_en.append(split_sent_en)
            sentence_list_fr.append(split_sent_fr)

    return sentence_list_en, sentence_list_fr

In [ ]:
sentences_en, sentences_fr = read_file('europarl/training.en','europarl/training.fr')

with open('europarl_split.en.pickle', 'wb') as f:
    pickle.dump(sentences_en, f)
    
with open('europarl_split.fr.pickle', 'wb') as f:
    pickle.dump(sentences_fr, f)

In [2]:
def train_sg_model(sentences, label):
    
    #https://machinelearningmastery.com/develop-word-embeddings-python-gensim/
    #https://radimrehurek.com/gensim/models/word2vec.html#id6
        
    # define training data
    model = Word2Vec(sentences, min_count = 2, sg = 1, negative = 5, iter=10)
    # summarize the loaded model
    print(model)
    # summarize vocabulary
    words = list(model_en.wv.vocab)
    print(words)
    # access vector for one word
    print(model['sentence'])
    # save model
    model.save_word2vec_format('model_'+label+'.bin')
    
    with open('words.'+label+'.pickle', 'wb') as f:
        pickle.dump(words,f)
    
#     # load model
#     loaded_model = Word2Vec.load('model_'+label+'.bin')
#     print(loaded_model)
    
    return model, words

In [ ]:
with open('europarl_split.en.pickle', 'rb') as f:
    sentences_en = pickle.load(f)
    
model_en, words_en = train_sg_model(sentences_en, 'en')

In [ ]:
with open('europarl_split.fr.pickle', 'rb') as f:
    sentences_fr = pickle.load(f)

model_fr, words_fr = train_sg_model(sentences_fr, 'fr')